In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import scipy.stats.stats as stats
import re
import math
from datetime import datetime
import importlib
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, LassoCV, RidgeClassifierCV
from sklearn.model_selection import cross_val_score, train_test_split, KFold, StratifiedKFold, GridSearchCV 
from sklearn.metrics import roc_auc_score, roc_curve, log_loss, f1_score, confusion_matrix, precision_score, recall_score, classification_report;
import time

import statsmodels.api as sm

In [3]:
# !pip install mlxtend
# !pip install lightgbm
# !pip install tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from autoscoring.AS import *
from autoscoring.AS_2 import *
from autoscoring.Binning import transform_df_to_woe, construction_binning, create_num_intervals, create_dict_cat_feats

In [5]:
df = pd.read_excel('data/ExcelSelfEmployed_rules.xlsx', sheet_name = 'Sheet1')

df['date_requested'] = pd.to_datetime(df['date_requested'])
DATE = 'date_requested'  # Дата, по которой аггрегировать и оценивать gini/psi и т.д. по дефолту дата выдачи.
df[DATE] = pd.to_datetime(df[DATE])

In [6]:
# В некоторых случаях при загрузке из excel файла может произойти проблема,
# связанная с тем, что в excel вещественные числа разделяются запятой,
# а в python разделение идёт по точке.
# Если возникла такая проблема, то можно заменить все запятые на точки и сделать преобразование типовю

# Запускать эту ячейку, только если есть данная проблема. Иначе можно пропустить.
'''
Запускать эту ячейку, только если есть данная проблема c приведением типво данных.
Иначе можно пропустить.

'''
for col in df.select_dtypes(include=object).columns:
    if col == DATE:
        continue
    df[col] = df[col].apply(lambda x: str(x).replace(',', '.'))

df = df.apply(pd.to_numeric, errors='ignore')
df[DATE] = pd.to_datetime(df[DATE], format='%Y-%m-%d')
df = df.replace('nan', np.nan)

'\nЗапускать эту ячейку, только если есть данная проблема c приведением типво данных.\nИначе можно пропустить.\n\n'

In [7]:
# задание названия переменной, являющейся таргетом
TARGET = 'npl1_1_7dpd'
df = df[df[TARGET].isna() == False]

# Фильтрация требуемого сегмента
df = df[(df['product_group'] == 'MICRO_SELF_EMPLOYED')]

df['target'] = df[TARGET]
y = df[TARGET]

In [8]:
# Filling missing values (np.nan, '_MISSING_')
cat_vars = df.select_dtypes(include=[object]).columns
df = filling(df)

min_num = 50
# Можно задать список признаков, который не будет учавствовать в предобработке
ignore_vars = ['date_requested', 'credit_id', 'target', DATE]
df, else_df = replace_not_frequent_2(df, cat_vars, num_min=min_num, ignore_vars=ignore_vars)

# Drop columns with only 1 value
df = drop_single_value_column(df, except_cols=ignore_vars)

In [9]:
# Удаляем лишние колонки, конкретный набор зависит от сэмпла.
cols_to_drop = [var for var in df.columns if 'SCOR' in var or 'street' in var or 'scoring' in var]
df.drop(cols_to_drop, axis=1, inplace=True)

#### Построение WOE таблицы

In [10]:
# Задаём набор признаков для формирования биннинга.
# Набор колонок, который не будет учавствовать в построении карты
technical_features = ['client_type2', 'credit_id',
                      'date_requested', 'target', 'npl1_1_7dpd', DATE]
technical_features = list(set(technical_features))
# Задаём набор признаков, которые будут учавствовать в биннинге,
# для этого удаляем все ненужные признаки из общего списка переменных.
features = df.drop(
    technical_features
, axis=1).columns

target_name = 'target'  # Наименование столбца - таргета
# Минимальное количество объектов в бине, по дефолту 0.05, требуется менять
# только в исключительных ситуациях
min_bin_size = 0.05
max_bin_count = 5  # Максимальное количество бинов при разбиении, по дефолту 5 - хороший вариант.
# Как определять WOE для пропусков, если их меньше чем min_bin_size:
# max - максимальное WOE
# max_cat - WOE самой крупной группы, можно оставить это по дефолту
# min - минимальное WOE
# zero - значение WOE становится 0
nan_to_woe = 'max_cat'
# Как определять WOE для _ELSE_ значений, если их меньше чем min_bin_size:
# max - максимальное WOE
# max_cat - WOE самой крупной группы, можно оставить это по дефолту
# min - минимальное WOE
# zero - значение WOE становится 0
else_to_woe = 'max_cat'
# Если True, то специально накладываются ограничения, чтобы WOE был монотонным по бинам,
# лучше оставить False, и потом в ручном режиме поменять, если не требуется обратного.
monotonic = False
n_jobs = 4  # Количество ядер для работы, чем больше тем быстрее, по дефолту 4.

iv_df, dropped_feats, feats, best_features, auto_woe = construction_binning(df, features, target_name,
                                                         max_bin_count=max_bin_count,
                                                         min_bin_size=min_bin_size,
                                                         nan_to_woe=nan_to_woe,
                                                         else_to_woe=else_to_woe,
                                                         monotonic=monotonic,
                                                         n_jobs=n_jobs)

# iv_df - таблицы WOE биннинга
# dropped_feats - набор переменных, для которых биннинг не посчитался,
# т.к. в переменной не нашлось сколько-нибудь хорошего разделения или признак практически константный
# feats - таблица полученная после предварительного отбора признаков, в которой указаны причины,
# по которым признак не очень хороший, можно игнорировать
# best_features - набор признаков после предварительного отбора внутри AutoML,
# можно использовать при дальнейшем отборе, можно проигнорировать
# auto_woe - техническая переменная, можно игнорировать

[LightGBM] [Info] Number of positive: 40, number of negative: 1352
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003346 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14809
[LightGBM] [Info] Number of data points in the train set: 1392, number of used features: 278
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.028736 -> initscore=-3.520461
[LightGBM] [Info] Start training from score -3.520461
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

### Вначале сделаем отбор переменных, далее будет подготовка признаков и построения дерева решений.

In [ ]:
# Набор обязательных признаков (для примера), которые должны остаться несмотря на отбор.
# Разкомментировать, если он нужен.

# mandatory_feats = ['credits_active_owner', 'credits_all_owner']
# mandatory_feats_woe = ['WOE_' + col for col in mandatory_feats]

In [11]:
# Порог для фильтрации по IV.
# Требуется устанавливать в зависимости от количества признаков, но лучше сильно не ограничивать.
# Например если признаков меньше 200, то можно поставить iv_co = 0.01 или 0.02.
# Иначе можно поставить iv_co = 0.03 - 0.05.
iv_co = 0.05
features_iv = iv_df[iv_df['IV'] > iv_co]['VAR_NAME'].unique()
print(len(features_iv))

# Добавялем к списку обязательные переменные, если требуется
# vars = add_mandatory_feats(vars, mandatory_feats)

IV = iv_df[['VAR_NAME', 'IV']].drop_duplicates()
IV = IV[IV['VAR_NAME'].isin(features_iv)]

139


In [12]:
# Данные подготовка нужна, чтобы быстрее сделать преобразование
iv_df = create_num_intervals(iv_df)
dict_cat_feats = create_dict_cat_feats(iv_df)

iv_co = None
X, y = transform_df_to_woe(df[features_iv], y, IV, iv_df, dict_cat_feats, iv_cut_off=iv_co, n_jobs=3)

Features left after IV drop: 139
DONE!


In [13]:
# Устанавливаем порого для фильтрации признаков по корреляциям.
# По дефолту 0.85 - оптимальное значение, в зависимости от требований
# можно поставить число в диапозоне 0.75 - 0.9.
cut_off = 0.85
X_train_corr = delete_correlated_features(X, cut_off=cut_off, is_plot_prev=False,
                                          exclude=[], IV_sort=True, iv_df=iv_df)

Dropped columns: [] 

Features left after correlation check: 139 

Not dropped columns: ['reapeted_app', 'days_from_fc_d_requested', 'days_from_last_c_d_requested', 'days_from_last_c_d_repaid', 'avg_expired_days', 'max_expired_days', 'penalty_cnt', 'count_repaid', 'early_repaid', 'credits_all_owner', 'mnt_outstand_balance', 'his_w_pay_st_by_act_crd_days', 'his_w_pay_st_by_act_crd_sum', 'his_w_pay_st_by_cls_crd_days', 'his_w_pay_st_by_cls_crd_sum', 'act_div_all_crd', 'sum_per_cred', 'h_cl_sum_div_days', 'h_ac_sum_div_days', 'cur_ac_sum_div_h_ac_sum', 'mnt_inst_owner_div_frst_cr', 'mnt_outstand_balance_div_frst_cr', 'fc_credits_active_owner', 'fc_credits_all_owner', 'fc_monthly_instalments_owner', 'fc_mnt_outstand_balance', 'fc_his_w_pay_st_by_act_crd_days', 'fc_his_w_pay_st_by_act_crd_sum', 'fc_his_w_pay_st_by_cls_crd_days', 'fc_his_w_pay_st_by_cls_crd_sum', 'fc_act_div_all_crd', 'fc_sum_per_cred', 'fc_h_cl_sum_div_days', 'fc_h_ac_sum_div_days', 'fc_cur_ac_sum_div_h_ac_sum', 'fc_h_ac_su

In [14]:
X = X[X_train_corr.columns]
X.columns = 'WOE_' + X.columns

In [15]:
# Фильтруем признаки по следующему принципу:
# для каждого признака делаем однофакторный расчёт gini на train выборке для каждого месяца
# в разбивке по DATE (обязательно должен присутствовать в df_train).
# После этого отфильтровываем признаки, для которых gini меньше чем gini_min
# для количества интервалов большего num_bad_intervals.
# Т.е. например, если gini_min=0.05 и num_bad_intervals=2, то если gini переменной больше 0.05 для всех месяцев, кроме одного - это ок. 
gini_min = 0.05  # 0.05 - по дефолту норм, можно взять в диапозоне 0.02 - 0.08 в зависимости от качества признаков.
num_bad_intervals = 2  # 2 месяца/недели по дефолту норм, можно взять 10-20% от общего количества месяцев в сэмпле.
date_name = DATE  # Дата, по которой делить на месяцы.
intervals = 'month'  # Интервалы времени для разбивки, month или week

gini_feats, df_gini_months = gini_month_selection(X, df, gini_min=gini_min,
                                                  num_bad_intervals=num_bad_intervals, date_name=date_name,
                                                  intervals=intervals)

In [16]:
# Отбираем признаки по их значимости во время построения Деревьев решений.
top_n = 50  # Количество лучшеих признаков, которые оставить. Можно брать в диапозоне 45-80.
rf_feats, rf_imp = rf_feature_selection(X[gini_feats], y, top_n=top_n)

In [17]:
# Отбираем признаки исходя из permutations_importance.
# Подробнее: https://www.kaggle.com/dansbecker/permutation-importance
top_n = 35  # Количество лучшеих признаков, которые оставить. Можно брать в диапозоне 30-50.
tf_feats, tf_imp = permutation_two_forest_selection(X[rf_feats], y, top_n=top_n)

In [18]:
# Итоговый набор переменных, который будет учавтсовать в построении дерева.
vars = [var.replace('WOE_', '') for var in tf_feats]

In [ ]:
# Заново добавляем список обязательных признаков.
# vars = add_mandatory_feats(vars, mandatory_feats)

In [19]:
vars

['fcb_first_cred_open_act',
 'fc_his_w_pay_st_by_cls_crd_sum',
 'bor_age',
 'mnt_outstand_balance',
 'average_monthly_income',
 'requested_amount_to_pay',
 'capsL1m',
 'average_monthly_income123',
 'sum_all_credit_sum',
 'cnt_IL_all',
 'work_amount',
 'fc_monthly_instalments_owner',
 'capsL3m',
 'fc_h_cl_sum_div_days',
 'DTI_11',
 'fcb_first_cred_open',
 'PTI_11',
 'fcb_last_open_d_cr_all',
 'cnt_IL_comp_no_Crd',
 'fc_act_div_all_crd',
 'last_deduction_date',
 'maritalStatus',
 'average_amount_opv',
 'fc_his_w_pay_st_by_act_crd_sum',
 'his_w_pay_st_by_cls_crd_sum',
 'pti7',
 'his_w_pay_st_by_act_crd_sum',
 'all_min_reg',
 'fc_credits_all_owner',
 'fc_h_ac_sum_div_days',
 'fc_mnt_outstand_balance',
 'requested_credit_count_days',
 'sum_groups_n',
 'min_all_planned_close_date_no_Crd',
 'fcb_date_diff']

### Делаем подготовку данных перед построением дерева

In [18]:
# !pip install category-encoders

In [20]:
import category_encoders as ce
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [21]:
df_base = df.copy()
# df = df_base.copy()

In [22]:
cat_feat = df[vars].select_dtypes(include=object).columns
num_feat = df[vars].select_dtypes(exclude=object).columns

In [23]:
# Составляем таблицы соответсвия при кодировани, чтобы можно было декодировать значения.
dict_cat_encoding = {}
for feat in cat_feat:
    target_encoder = ce.target_encoder.TargetEncoder(smoothing=0.1)
    target_encoder.fit(df[[feat]], df[TARGET])

    values = df[[feat]].drop_duplicates().copy()
    encoding = target_encoder.transform(values)
    encoding.columns = [f'{feat}_enc']

    dict_cat_encoding[feat] = pd.concat([values, encoding], axis=1).reset_index(drop=True)

# Кодируем категориальный переменные.
target_encoder = ce.target_encoder.TargetEncoder(smoothing=0.1)
df[cat_feat] = target_encoder.fit_transform(df[cat_feat], df[TARGET])

##### Заполняем NaN

Есть 3 варианта заполнения:
* Явно задать значения для NaN различных признаков
* Медианным значением
* Заполнить автоматически наиболее подходящим с точки зрения дефолтности значением
Явно задем значение для NaN

<b>Требутся выбрать одно из 3ех, лучше всего последнее при прочих равных.</b>

Явно задаем значение для NaN

In [ ]:
# Требуется явно задать значения для NaN
dict_nan_encoding = {
    # '<Наименование признака>': <значение, на которое делать замену NaN>,
}
for feat, val in dict_nan_encoding.items():
    df[feat].fillna(val, inplace=True)

Заполняем медианой

In [ ]:
dict_nan_encoding = {}
# Заполняем NaN значения медианой данного количественного признака
for feat in num_feat:
    if dict_nan_encoding.get(feat) is None:
        dict_nan_encoding[feat] = df[feat].median()
    df[feat].fillna(df[feat].median(), inplace=True)

Автоматический подбор

In [24]:
# Автоматически ищем наиболее подходящую замену NaN с точки зрения дефолтности сегментов.
dict_nan_encoding = {}

for feat in num_feat:
    if df[feat].isna().sum() == 0:
        continue

    d3 = iv_df[iv_df['VAR_NAME'] == feat]
    dr_nan = d3.loc[d3['MIN_VALUE'].isna() == True, 'DR'].values[0]

    ind_for_nan = np.argmin(np.abs(d3[d3['MIN_VALUE'].isna() != True]['DR'] - dr_nan))
    segment_for_nan = d3[d3['MIN_VALUE'].isna() != True].iloc[ind_for_nan]

    value = df[(df[feat] > segment_for_nan['MIN_VALUE']) & (df[feat] < segment_for_nan['MAX_VALUE'] + 0.001)][feat].median()

    if dict_nan_encoding.get(feat) is None:
        dict_nan_encoding[feat] = value
    df[feat].fillna(value, inplace=True)

Сохраняем метод заполнения NaN и кодировку категориальных переменных в excel

In [77]:
name = 'result_rules/encoding_methods'  # Наименование файла, куда сохранять информацию
save_encoding_excel(dict_cat_encoding, dict_nan_encoding, name=name)

### Строим дерево решений

В каждом прямоугольнике расположена следующая информация:
* решающее правило для разделения на следующем уровне, например: fcb_first_cred_open <= 83.0
* количество объектов в данном узле: samples = 1740
* размеры классов: value = [1688 (количество класса 0), 52 (количество класса 1)]

Далее на следующей уровне будет разделение по текущему правилу. Объекты, удовлетворяющие ему пойдут влево, другие пойдут вправо.

Так после правила "fcb_first_cred_open <= 83.0" влевом узле будут объекты, для которых это верно (value=[48, 12]), остальные пойдут в правый узел (value=[1640, 40]).

In [ ]:
# Функция для построения и визуализации нескольких деревьев решений.
# Параметры:
#     Обязательные:
#         - df (DataFrame): Исходный DataFrame, содержащий данные для обучения модели.
#         - vars (list): Список признаков, используемых для обучения модели.
#         - TARGET (str): Имя целевой переменной.
#     Опциональные:
#         - num_forests (int): Количество деревьев с разными случайными наборами признаков. По умолчанию 1.
#         - min_samples_leaf (int): Минимальное количество объектов в одном листе дерева. По умолчанию 60.
#         - max_depth (int): Максимальная глубина дерева. По умолчанию 3.
#         - percent_feat (float): Процент признаков, который брать для построения нового дерева. По умолчанию 1.0.
#         - pic_name (str): Путь и имя для сохранения изображения. По умолчанию 'result_rules/example_tree_structure_0_v'.
#         - show_image (bool): Параметр для отображения или не отображения картинки. По умолчанию True.
#         - full_info (bool): Параметр для вывода полной информации в узлах или только условие, Gini, Samples, Value. По умолчанию True.

visualize_decision_trees(df, vars, TARGET, num_forests=4,
                        min_samples_leaf=60, max_depth=3, percent_feat=0.7,
                        pic_name='result_rules/example_tree_structure2_v.png', 
                        show_image=True, full_info=True)

In [26]:
num_forests = 4  # Количество деревьев с разным случайным набором признаков.
min_samples_leaf = 60  # Минимальное количество объектов в одном листе дерева.
max_depth = 3  # Максимальная глубина дерева.
percent_feat = 0.7  # Процент признаков, который брать для построения нового дерева.
pic_name = 'result_rules/example_tree_structure2_v.png'  # Где и под каким именем сохранять картинки


for i in range(1, num_forests+1):
    __vars = [np.random.choice(vars) for i in range(int(len(vars) * percent_feat))]
    clf_tree = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=142)
    clf_tree.fit(df[__vars], df[TARGET])

    fig = plt.figure(figsize=(100,80))
    _ = tree.plot_tree(clf_tree, 
                    feature_names=__vars,
                    #  class_names=['good', 'bad'],
                    #    filled=True
                    )
    fig.savefig(pic_name[:-4] + str(i) + pic_name[-4:])
    # plt.show()  # чтобы отрисовывать деревья
    plt.close()  # чтобы не отрисовывать деревья